In [1]:
import time
from multiverse import MeasurementPlaneClient

def on_result_callback(result):
    print(f"New Result Received: {result}")

def on_completion_callback(status):
    print(f"Measurement Completed with Status: {status}")

BROKER_URL = "http://129.6.254.164:5672/"
# Step 1: Get the capabilities published by agents
mpClient = MeasurementPlaneClient(BROKER_URL)

Give it some time to start loading the available capabilities

In [3]:
capabilities = mpClient.get_capabilities(capability_type="measure")
print("Available Capabilities:", capabilities)

Available Capabilities: {'f7fd9c11ab291bc861450dda33a253b4dfa9e8a7078ead309ea584839573f746': {'label': 'Timetags measurement', 'endpoint': '/multiverse/qnet/tt/Alice', 'schedule': 'now', 'capabilityName': 'TimetagsMeasurementAlice', 'parameters': {'type': 'object', 'properties': {'channels': {'description': 'Channels numbers to measure', 'type': 'array', 'items': {'type': 'number'}}}, 'required': ['channels']}, 'resultSchema': {'type': 'object', 'patternProperties': {'^[0-9]+$': {'type': 'object', 'properties': {'WR_time': {'type': 'array', 'items': {'type': 'string'}}}, 'required': ['WR_time']}}, 'additionalProperties': False}, 'timestamp': '2024-06-25 18:33:23.63', 'nonce': 'string', 'metadata': 'object', 'capability': 'measure'}}


In [5]:
# Step 2: Pick up one capability (by endpoint and name)
endpoint = "/multiverse/qnet/tt/Alice"  # Replace with an actual agent endpoint
capability_name = "TimetagsMeasurementAlice"  # Replace with an actual capability name

capability_id = mpClient.calculate_capability_id({"endpoint":endpoint, "capabilityName": capability_name})
if capability_id in capabilities:
    capability = capabilities[capability_id]
    print(f"Selected Capability: {capability}")
else:
    capability = None

if not capability:
    print(f"Capability not found.")


Selected Capability: {'label': 'Timetags measurement', 'endpoint': '/multiverse/qnet/tt/Alice', 'schedule': 'now', 'capabilityName': 'TimetagsMeasurementAlice', 'parameters': {'type': 'object', 'properties': {'channels': {'description': 'Channels numbers to measure', 'type': 'array', 'items': {'type': 'number'}}}, 'required': ['channels']}, 'resultSchema': {'type': 'object', 'patternProperties': {'^[0-9]+$': {'type': 'object', 'properties': {'WR_time': {'type': 'array', 'items': {'type': 'string'}}}, 'required': ['WR_time']}}, 'additionalProperties': False}, 'timestamp': '2024-06-25 18:33:23.63', 'nonce': 'string', 'metadata': 'object', 'capability': 'measure'}


In [6]:
# Step 3: Instantiate a specification of a measurement from the capability
measurement = mpClient.create_measurement(capability)
print(f"Created Measurement: {measurement.specification_message}")

# Step 4: Configure the measurement with parameters
schedule = "now"  # Replace with your schedule: `now / period`, `start ... stop / period`, etc.
parameters = {
    "channels": [1,2]
    # Add more parameters based on the capability.parameters JSON schema
}
measurement.configure(
    schedule=schedule,
    parameters=parameters,
    stream_results=True,  # Stream results
    redirect_to_storage=True,  # Redirect to storage
    result_callback=on_result_callback,  # Callback function for new results
    completion_callback=on_completion_callback  # Callback function for measurement completion
)
print(f"Configured Measurement Specification: {measurement.specification_message}")

# Step 5: Send the Measurement
mpClient.send_measurement(measurement)

# Simulate some wait time for the measurement to finish
time.sleep(5)  # Wait for 5 seconds (replace with appropriate wait time)

# Step 6: Interrupt the measurement
mpClient.interrupt_measurement(measurement)
print(f"Measurement Interrupted")
print(f"Measurement Results: ", measurement.results)


Created Measurement: {'label': 'Timetags measurement', 'endpoint': '/multiverse/qnet/tt/Alice', 'schedule': 'now', 'capabilityName': 'TimetagsMeasurementAlice', 'parameters': {'type': 'object', 'properties': {'channels': {'description': 'Channels numbers to measure', 'type': 'array', 'items': {'type': 'number'}}}, 'required': ['channels']}, 'resultSchema': {'type': 'object', 'patternProperties': {'^[0-9]+$': {'type': 'object', 'properties': {'WR_time': {'type': 'array', 'items': {'type': 'string'}}}, 'required': ['WR_time']}}, 'additionalProperties': False}, 'timestamp': '2024-06-25 18:33:23.63', 'nonce': 'string', 'metadata': 'object', 'specification': 'measure'}
Configured Measurement Specification: {'label': 'Timetags measurement', 'endpoint': '/multiverse/qnet/tt/Alice', 'schedule': 'now', 'capabilityName': 'TimetagsMeasurementAlice', 'parameters': {'channels': [1, 2]}, 'resultSchema': {'type': 'object', 'patternProperties': {'^[0-9]+$': {'type': 'object', 'properties': {'WR_time':

New Result Received: ['1719355052.7794628']
